In [1]:
import bs4
import requests as r
from IPython.display import Javascript
import pprint
import pandas as pd
from datetime import date

def open_web(URL):
    display(Javascript('window.open("{url}");'.format(url=URL)))

In [2]:
PAGE = 1
URL_prefix = "https://www.strumentimusicali.net/product_info.php/products_id/"

def get_url_from_page(PAGE):
    return "https://www.strumentimusicali.net/advanced_search_result.php?sort=3d&categories_id=21&inc_subcat=1&keywords=&page=" + str(PAGE) + "&products_available=1&used=1"
def get_soup(url):
    response = r.get(url)
    response.raise_for_status()
    return bs4.BeautifulSoup(response.text, "html.parser")
def find_firstGrade_characteristics(soup):
    to_ret = []
    for child in soup.descendants:
        if child.name == 'li':
            for child2 in child.descendants:
                if (child2.name == 'b' or child2.name == 'strong') and ':' in child2.text:
                    to_ret.append(child2.text)
    return to_ret


soup = get_soup(get_url_from_page(PAGE))
annunci = soup.find_all('tr', class_='productListing-even')

names = []
prices = []
links = []
descriptions = []
characteristics = []

while (len(annunci) > 0):

    for annuncio in annunci:

        title = annuncio.find('b').text.strip()
        link = annuncio.find('a')['href']
        price = annuncio.find('span', class_='d-block fontSize14 marginBottom5 bold')

        if price is None: # sconto
            price = annuncio.find('span', class_='productSpecialPrice bold')
        price = price.text

        if URL_prefix in link:
            names.append(title)
            prices.append(price)
            links.append(link)
            firstGrade_soup = get_soup(link)
            firstGrade_page = firstGrade_soup.find('td', class_='productDesc')
            descriptions.append(firstGrade_page.text.replace('\n', ' ').replace('       ', ' ').strip())
            characteristics.append(find_firstGrade_characteristics(firstGrade_soup))

    PAGE += 1
    soup = get_soup(get_url_from_page(PAGE))
    annunci = soup.find_all('tr', class_='productListing-even')

products = {'Titolo Annuncio': names,
        'Prezzo': prices,
        'Descrizione': descriptions,
        'Link': links,
        'Caratteristiche': characteristics
        }
df = pd.DataFrame(products)
df.to_csv("usato"+str(date.today())+".csv", index=False)